In [1]:
from src.vectorstore import connect_to_milvus, activate_collection
from src.retrievers import MilvusRerankRetriever

import os
from dotenv import load_dotenv

/Users/damir_bogdan/Library/Caches/pypoetry/virtualenvs/huberman-rag-09zSrCFX-py3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching 30 files: 100%|██████████| 30/30 [00:00<00:00, 260515.78it/s]


In [2]:
from src.retrievers import MilvusMultiVectorRetriever

from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

import os
from dotenv import load_dotenv

In [4]:
connect_to_milvus()
collection = activate_collection()
collection.load()

In [5]:
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
COHERE_API_KEY = os.getenv("COHERE_API_KEY")

In [7]:
# Template

"""If you do not have the answer in the CONTEXT write "I don't know".
Do not make things up. Do not fabricate facts. """
RAG_TEMPLATE = """\
You are a Huberman Lab podcast listener. 
You will be provided a CONTEXT of the podcasts and a QUESTION.
Based on the CONTEXT answer the QUESTION.

CONTEXT:
{context}

QUESTION:
{question}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_TEMPLATE)

# Setup a chat model
chat_model = ChatOpenAI()
# Setup a retriever
retriever = MilvusMultiVectorRetriever()

# Setup a chain
multivector_chain = (
    # INVOKE CHAIN WITH: {"question" : "<<SOME USER QUESTION>>"}
    # "question" : populated by getting the value of the "question" key
    # "context"  : populated by getting the value of the "question" key and chaining it into the base_retriever
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    # "context"  : is assigned to a RunnablePassthrough object (will not be called or considered in the next step)
    #              by getting the value of the "context" key from the previous step
    | RunnablePassthrough.assign(context=itemgetter("context"))
    # "response" : the "context" and "question" values are used to format our prompt object and then piped
    #              into the LLM and stored in a key called "response"
    # "context"  : populated by getting the value of the "context" key from the previous step
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

In [8]:
res = multivector_chain.invoke({"question" : "What is placebo?"})

In [9]:
res

{'response': AIMessage(content='A placebo is an inert substance or treatment that is used in experiments or clinical trials to compare the effects of a drug or treatment solution. It is used to measure the expectation of what a drug treatment will do, and it can lead to real biological changes in the body and brain.', response_metadata={'token_usage': {'completion_tokens': 56, 'prompt_tokens': 2833, 'total_tokens': 2889}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-9a86595e-aa22-488d-a11a-1f9e72684896-0'),
 'context': [[Document(page_content="Simply put, a placebo drug,  again, a drug that has no direct action  on the dopaminergic system,  if it's given to somebody who has Parkinson's or who doesn't  and they are told this drug is going to increase levels  of dopamine in your brain  and potentially improve your symptoms of Parkinson's  or have some other effect,  well, it succeeded in increasing dopamine levels  within 